# Part 1

Note to grader: For my responses, I will be using the abbreviations SW for Smith-Waterman and NW for Needleman-Wunsch. For the alignment matrices, the match/mismatch matrix is denoted as m, seq1 gap matrix as ix, and the seq2 gap matrix as iy.

Question 1:

a) For initialization, both SW and NW require a match/mismatch score matrix, gap opening penalty, gap extension penalty, alignment matrices (m, ix, iy), and pointer matrices for the traceback in order to traverse through the alignment matrices to find the optimal alignment. They also require two sequences that the user wishes to align. For execution, they require the matrices to be initialized and a dynamic programming approach for incrementally filling them in. Then, a traceback start point is used in conjunction with the matrices to traceback through the matrices to find the alignment. For termination, they require an end condition for the traceback.

b) For both, when the align() method is called, the alignment is returned. For SW, this is a local alignment of the two sequences. For NW, this is a global alignment of the two sequences. Gaps in the alignment are represented using dashes (“-”). In my implementation, the alignment score is also accessible as a class attribute (or using the score() method).

c) For both, there are two main parts to the algorithm: filling out the matrices and doing the traceback. For this question, let m = length(seq1) and n = length(seq2). The runtime complexity of the dynamic programming approach to fill out the alignment matrices is O(m*n) because of the nested for loops. The runtime complexity of the traceback is O(n) because you just go through the alignment once. The overall runtime complexity of the algorithms is the highest of the runtime complexities of the parts. So for both SW and NW, the runtime complexity is O(m*n).

Question 2:

Both require alignment parameters (score matrix & gap penalties) for initialization. Both require sequences to be inputted (as strings in my implementation). Both have three alignment matrices, but those matrices are initialized differently between the two algorithms. For SW, the top row and the left column of m is initialized to 0. The top row and left column of ix and iy are initialized to -inf. For NW, the (0,0) cell of m is initialized to 0. The left column of the ix initialized to: gap opening penalty + (gap extension penalty * row number). The top row of iy is initialized to: gap opening penalty + (gap extension penalty * column number). For both, all other cells in the top row and left column are initialized to -inf. For both, I had 3 pointer matrices (one for each alignment matrix).

For execution, the recurrence relations to fill out the alignment matrices are virtually identical. The only difference is that for SW, when evaluating each cell in m, an additional option is added: 0. Since the recurrence relation finds the max of the score options for that cell and 0 is now also an option, this ensures that there are no negative values in m for SW. For both NW and SW, as the matrices are filled out using dynamic programming, the origin of each score is stored in pointer matrices to enable traceback.

For SW, the traceback starts at the maximum match/mismatch matrix value. For NW, the traceback starts at the bottom right cell of the matrix with the highest value there. For both, the traceback continues through the alignment score matrices using the pointer matrices as guidance about where to go next from each cell. For termination, the SW traceback ends when the traceback hits a 0 value. For NW, the traceback ends when the traceback hits the top left corner.

Question 3:

Affine-gap alignment requires 3 alignment matrices, whereas linear-gap alignment only requires 1. For affine-gap alignment, this means that when filling out each cell in each matrix, values from cells in other matrices are used. Additionally, for linear-gap alignment, only 1 pointer matrix is required. For affine-gap alignment, the pointers are a bit more complicated since they refer to both cell locations and matrix types. There are many ways to implement this. For my implementation I had 3 pointer matrices (one for each alignment matrix) that point to the type (m, ix, iy) that the current cell value in the alignment matrix came from. For linear-gap alignment, the traceback walks through a single alignment matrix. For affine-gap alignment, the traceback can walk through all three alignment matrices.

Question 4:

My API is in my README.md in my github repo.

Question 5:

I implemented overlap alignment. There is also a unit test for it (both aligning & scoring).

# Part 2

In [1]:
# Import classes
from align import algs

### Question 1

In [ ]:
# Load fasta file names for alignments
def load_pairs(pairs_txt_file):
    # Read in pairs
    with open(pairs_txt_file) as pf:
        # Read in each line, remove whitespace, and make list of tuples representing each pair
        pairs_files = [tuple(line.strip().split()) for line in pf]
    return pairs_files

# Load sequences given fasta file names
def load_sequences(fasta_file_names):
    pa = algs.PairwiseAligner("BLOSUM50", -3, -1)
    sequence_pairs = [tuple([pa.load_fasta(f1), pa.load_fasta(f2)]) for (f1, f2) in fasta_file_names]
    return sequence_pairs

def get_q1_scores(allpairs):
    sw = algs.SmithWaterman("BLOSUM50", -11, -3)
    sw.load_scoring_matrix()
    scores = []
    for (s1, s2) in allpairs:
        s = sw.score(s1, s2)
        scores.append(s)
    return scores

# Read in names of fasta files for true & false alignments
pospairs_files = load_pairs("scoring_matrices/Pospairs.txt")
negpairs_files = load_pairs("scoring_matrices/Negpairs.txt")
# Read in sequences from those fasta files
pospairs = load_sequences(pospairs_files)
negpairs = load_sequences(negpairs_files)
allpairs = pospairs + negpairs
# Get alignment scores
q1_alignment_scores = get_q1_scores(allpairs)
with open("part2_results_data/q1.txt", "w") as q1:
    q1.writelines("%s\n" % score for score in q1_alignment_scores)